### Sentiment analysis of Tweets

### libraries 

In [47]:
import pandas as pd 
import numpy as np 
import nltk
from nltk.stem import PorterStemmer
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec,KeyedVectors
import gensim
import warnings
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

warnings.filterwarnings(action='ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohan.sharma2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1. Data cleaning and preprocessing 
2. Applying stopwords, word2vec and avgword2vec
3. Spliting the data into train and test.
4. Applying Ml algo to prediction.
5. Analyzing the results.

#### Reading the data and basic data understanding using visualizations 

In [22]:
headers = ["ID", "Code", "Sentiment", "Comment"]

# Read the CSV without headers and assign your custom headers
df = pd.read_csv(r"C:\Users\rohan.sharma2\Downloads\archive\twitter_training.csv", header=None, names=headers)
df.head(2)

,ID,Code,Sentiment,Comment
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...


 ##### Dropping Code and ID as they are irrelevant to the code

In [23]:
df.drop(columns=['Code','ID'],inplace=True)

In [24]:
df['Sentiment'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [25]:
df = df[df['Sentiment'] != 'Irrelevant']

#### Here we have 3 different values in sentiments need to change it to ints for ML models to train easily

In [26]:
df['Sentiment']=df['Sentiment'].replace({'Negative': 0,'Positive': 2,'Neutral':1})

#### Need to now process the comments 

In [27]:
ps = PorterStemmer()
df['stemmed_comments'] = df['Comment'].apply(
    lambda x: ' '.join([
        ps.stem(word) for word in re.sub(r'\S+@\S+\.\S+|https?://\S+|www\.\S+|[^a-zA-Z\s]', '', str(x)).split()
    ])
)


In [28]:
X= df['stemmed_comments']

In [29]:
y=df['Sentiment']

In [30]:
stop_words = stopwords.words('english')


In [31]:
def remove_stopwords(text):
    """
    Remove stopwords from text data.

    This function filters out common stopwords from the text data. 
    Stopwords are removed based on the NLTK's English stopwords list.

    Parameters:
    text (pandas.Series): A pandas Series containing text data.

    Returns:
    pandas.Series: A pandas Series with stopwords removed from the text.
    """
    # Removing stopwords
    text = text.apply(lambda x: " ".join(word for word in str(x).split() if word not in stop_words))
    return text

In [32]:
remove_stopwords(X)

0                                 im get borderland murder
1                                         come border kill
2                                   im get borderland kill
3                                im come borderland murder
4                                 im get borderland murder
                               ...                        
74677    realiz window partit mac like year behind nvid...
74678    realiz mac window partit year behind nvidia dr...
74679    realiz window partit mac year behind nvidia dr...
74680    realiz window partit mac like year behind nvid...
74681    like window partit mac like year behind driver...
Name: stemmed_comments, Length: 61692, dtype: object

In [37]:
### Fx to convert the words into vectors
def vectorize_comment(comment):
    words = [word for word in comment if word in model.wv]
    return np.mean(model.wv[words], axis=0) if words else np.zeros(model.vector_size)

# Vectorize all comments
xtrain_vectors = np.array([vectorize_comment(comment) for comment in X])

In [38]:
model = Word2Vec(sentences=X, vector_size=100, window=5, min_count=1, workers=4)

In [39]:
ytrain = y
X_train, X_test, y_train, y_test = train_test_split(xtrain_vectors, ytrain, test_size=0.2, random_state=42)

In [42]:
X_train

array([[ 0.08688786, -0.04517177,  0.1122427 , ...,  0.06302276,
        -0.02525851,  0.15368578],
       [ 0.08679375, -0.08679009,  0.06925038, ...,  0.08986504,
         0.01115826,  0.18043676],
       [ 0.14181997, -0.09293198,  0.06384929, ...,  0.05784483,
         0.02776417,  0.16949186],
       ...,
       [ 0.12369649, -0.06444271,  0.07315879, ...,  0.0756216 ,
        -0.01237735,  0.18005913],
       [ 0.14492117, -0.07849602,  0.06105435, ...,  0.07141475,
         0.00805079,  0.17611519],
       [ 0.14297166, -0.09591766,  0.07622663, ...,  0.05141467,
        -0.00951197,  0.15359309]])

In [44]:
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [48]:
y_pred = model_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6896831185671448


In [49]:
sentiment_analysis= MultinomialNB()


In [ ]:

.fit(X_train,y_train)


In [19]:
model2 = gensim.models.Word2Vec(X_train, min_count=1, vector_size=100,window=5, sg=1)